In [ ]:
import pandas as pd
sample_df = pd.read_csv("../data/medQA_openanswers.csv")[:25]

In [ ]:
sample_df.head(10)

In [ ]:
from tech_train_functions import is_similar
sample_df.loc[:,"is correct reply"] = sample_df.apply(lambda row:row["answer"].lower() in row["reply_open"].lower(), axis=1)

In [ ]:
sample_df.head(10)

In [ ]:
sample_df["is correct reply"].value_counts()

In [ ]:
import os
from tech_train_functions import analyze_TA4H
ta4h_endpoint=os.getenv("TA4H_ENDPOINT")
key=os.getenv("TA4H_KEY")

In [ ]:
def is_similar_entities(gold_answers, answers):
    gold_answer_entities_list = analyze_TA4H(ta4h_endpoint, key, gold_answers)
    print(gold_answer_entities_list)
    gold_answers_entities_short = [[entities_format(e) for e in gold_answer_entities] for gold_answer_entities in gold_answer_entities_list]
    print(gold_answers_entities_short)
    answer_entities_list = analyze_TA4H(ta4h_endpoint, key,answers)
    answer_entities_short = [[entities_format(e) for e in answer_entities] for answer_entities in answer_entities_list]
    print(answer_entities_short)
    return compare_lists(gold_answers_entities_short, answer_entities_short)

def entities_format(e):
   return e["normalized_text"] if e["normalized_text"] else e["text"].lower()

def compare_lists(list1, list2):
    # Initialize an empty list to store the results
    result = []

    # Iterate over each pair of lists
    for sublist1, sublist2 in zip(list1, list2):
        # Check if there is no intersection between the two sublists
        non_covered_entities = set(sublist1)-set(sublist2)
        result.append(1 if len(non_covered_entities)==0 else 0)

    return result

In [ ]:
sample_df["is correct entities"] = is_similar_entities(sample_df["reply_open"].tolist(), sample_df["answer"].tolist())

In [ ]:
sample_df